# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [12]:
#df.head()
df.tail()

,Product,Review
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \r\n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [6]:
#!pip install langchain_community

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7, )
#0.7 is a good a bit more creative temperature value

In [9]:
prompt = ChatPromptTemplate.from_template( 'the reviews are summarised for every {product}'
 
)

In [10]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\rospi\AppData\Local\Temp\ipykernel_2500\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [28]:
product = 'door'
chain.run(product)

"1. Door A: Customers love the sleek design and easy installation process of Door A. They appreciate its durability and security features.\n\n2. Door B: Reviewers are impressed with the high-quality materials used in Door B and its energy efficiency. Some users noted that the door can be difficult to install.\n\n3. Door C: Users enjoy the modern and stylish look of Door C, but some have experienced issues with the door's locking mechanism. Overall, customers are satisfied with the door's performance.\n\n4. Door D: Customers appreciate the versatility of Door D, as it can be customized to fit different styles and sizes of homes. However, some users have reported issues with the door's durability over time."

## SimpleSequentialChain

In [29]:
from langchain.chains import SimpleSequentialChain

In [30]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    'the reviews are summarised for every {product}'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [31]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Translate the following summary to Italian:\n{text}'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [32]:


overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [33]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Unfortunately, I do not have access to the specific reviews for every door.
Purtroppo, non ho accesso alle recensioni specifiche per ogni porta.

> Finished chain.


'Purtroppo, non ho accesso alle recensioni specifiche per ogni porta.'

**Repeat the above twice for different products**

## SequentialChain

In [34]:
from langchain.chains import SequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  'translate into chinese'
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= 'chinese'
                    )


In [40]:
second_prompt = ChatPromptTemplate.from_template(
    'it should be maximum 20 words'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= 'summary'
                    )


In [43]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    'in french please'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key= 'French Translation'
                      )


In [54]:

#prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        'please rewrite the summary in a funny way')
# Set the output_key to a valid string, e.g., 'funny_summary'

chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='funny_summary'
                         )

In [57]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["input"],
    output_variables=["chinese", "summary", "French Translation", "funny_summary"],
    verbose=True
)

In [58]:
review = df.Review[5]
overall_chain(review)

C:\Users\rospi\AppData\Local\Temp\ipykernel_2500\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'input': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'chinese': '翻译成中文',
 'summary': 'Keep your message concise and to the point - 20 words is the limit for effective communication.',
 'French Translation': "en français s'il vous plaît",
 'funny_summary': 'Get ready to laugh your socks off with this hilarious summary! Dive into a wild adventure filled with comedy gold as we follow the antics of a wacky cast of characters and their ridiculous escapades. From silly mishaps to laugh-out-loud moments, this summary will have you in stitches from start to finish. So grab your popcorn and get ready for a side-splitting ride through this uproarious story!'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [59]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [60]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [61]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [62]:
llm = ChatOpenAI(temperature=0)

In [63]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [64]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [65]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [66]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [67]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\rospi\AppData\Local\Temp\ipykernel_2500\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [68]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities, which depends only on the temperature of the body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is proportional to the frequency of the radiation and the temperature of the body."

In [72]:
chain.run("what is Genome")



> Entering new MultiPromptChain chain...
biology: {'input': 'what is Genome'}
> Finished chain.


"A genome is the complete set of genetic material (DNA or RNA) present in an organism. It includes all of the organism's genes, as well as non-coding sequences of DNA that regulate gene expression and other genetic elements. The genome contains the instructions for building and maintaining an organism, and it plays a crucial role in determining an organism's traits and characteristics. Genomes can vary in size and complexity between different species, and studying genomes can provide valuable insights into the genetic basis of traits, evolution, and disease."

In [73]:
chain.run("Which one is the worst sickness in the world?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Which disease is considered the most severe in the world?'}
> Finished chain.


'It is difficult to pinpoint one specific disease as the most severe in the world, as the severity of a disease can vary depending on factors such as access to healthcare, treatment options, and individual susceptibility. However, some of the most severe diseases in terms of global impact and mortality rates include HIV/AIDS, tuberculosis, malaria, and certain types of cancer. These diseases have significant impacts on public health and require ongoing research and intervention efforts to control and prevent their spread.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**